In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout, PReLU, BatchNormalization, ELU, GaussianNoise, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
import gc
import os
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest
#ARI: Arizona Cardinals
#ATL: Atlanta Falcons
#BAL: Baltimore Ravens
#BUF: Buffalo Bills
#CAR: Carolina Panthers
#CHI: Chicago Bears
#CIN: Cincinnati Bengals
#CLE: Cleveland Browns
#DAL: Dallas Cowboys
#DEN: Denver Broncos
#DET: Detroit Lions
#GB: Green Bay Packers
#HOU: Houston Texans
#IND: Indianapolis Colts
#JAX: Jacksonville Jaguars
#KC: Kansas City Chiefs
#MIA: Miami Dolphins
#MIN: Minnesota Vikings
#NE: New England Patriots
#NO: New Orleans Saints
#NYG: New York Giants
#NYJ: New York Jets
#OAK: Oakland Raiders
#PHI: Philadelphia Eagles
#PIT: Pittsburgh Steelers
#SD: San Diego Chargers
#SEA: Seattle Seahawks
#SF: San Francisco 49ers
#STL: Saint Louis Rams
#TB: Tampa Bay Buccaneers
#TEN Tennessee Titans
#WAS: Washington Redskins

In [3]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

# fill null
    df = df.fillna(df.median())

# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    df['Cleaned_Turf'] = df['Turf'].map(Turf)

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = abs(df['Degree_to_Rusher'] - df['Rusher_Dir_std'])
    #back_direction
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium clean
    map_stad = {'Broncos Stadium at Mile High': 'Broncos Stadium At Mile High', 'CenturyField': 'CenturyLink Field', 'CenturyLink': 'CenturyLink Field', 'Everbank Field': 'EverBank Field', 'FirstEnergy': 'First Energy Stadium', 'FirstEnergy Stadium': 'First Energy Stadium', 'FirstEnergyStadium': 'First Energy Stadium', 'Lambeau field': 'Lambeau Field', 'Los Angeles Memorial Coliesum': 'Los Angeles Memorial Coliseum', 'M & T Bank Stadium': 'M&T Bank Stadium', 'M&T Stadium': 'M&T Bank Stadium', 'Mercedes-Benz Dome': 'Mercedes-Benz Superdome', 'MetLife': 'MetLife Stadium', 'Metlife Stadium': 'MetLife Stadium', 'NRG': 'NRG Stadium', 'Oakland Alameda-County Coliseum': 'Oakland-Alameda County Coliseum', 'Paul Brown Stdium': 'Paul Brown Stadium', 'Twickenham': 'Twickenham Stadium'}
    
    for stad in df['Stadium'].unique():
        if stad in map_stad.keys():
            pass
        else:
            map_stad[stad]=stad

    df['Stadium'] = df['Stadium'].map(map_stad)

# wind 
    def give_me_WindSpeed(x):
            x = str(x)
            x = x.replace('mph', '').strip()
            if '-' in x:
                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
            elif 'gusts up to' in x:
                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
            elif 'clam' in x:
                x = 0
            try:
                return float(x)
            except:
                return -99
    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

    
# speed
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]

    
    return df

In [5]:
train = data_clean(train_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


In [92]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# handoff times
    df1 = df1.sort_values(['GameId','PlayId','Quarter','GameClock'])
    df1['# of Handoff Play'] = 1
    df1['# of Handoff Play'] = df1[['GameId','PlayId','# of Handoff Play']].groupby(['GameId','PlayId']).cumsum()
# min_time_to_tacke
    df3['Min_Time_Tacke'] = df3['Distance_to_Rusher']/df3['S']
    df3.loc[df3['Min_Time_Tacke'] == np.inf, 'Min_Time_Tacke'] = 20
# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])

# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 '# of Handoff Play','Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Offense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std','num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff',\
                 'run_def']]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Cleaned_GameWeather','Humidity','Temperature', \
              'Week']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','New_Y'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke']+list(range(2))]
    defender = defender.sort_values(['PlayId','New_Y'])
    
    y = df1[['PlayId','Yards']].sort_values('PlayId').drop('PlayId',axis = 1)
#get_dummy
    def encoding_cat(df):
        for cat in df.dtypes[df.dtypes=='object'].index.tolist():
            a = pd.get_dummies(df[cat])
            df = df.drop(cat,axis = 1)
            df = pd.concat([df,a],axis = 1)
        return df
    rusher = encoding_cat(rusher)
    game = encoding_cat(game)
    offender = encoding_cat(offender)
    defender = encoding_cat(defender)
    
    return rusher, game, offender, defender, y

In [93]:
rusher, game, offender, defender, y = split_data(train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [94]:
defender

,PlayId,PlayerAge,PlayerHeight,PlayerWeight,New_X,New_Y,Orientation_std,Dir_std,back_oriented_down_field,Horizontal Speed,Vertical Speed,S,A,Dis,Distance_to_Rusher,Degree_to_Rusher,Degree_Diff,Min_Time_Tacke,0,1,C,CB,DB,DE,DL,DT,FB,FS,G,ILB,LB,MLB,NT,OLB,OT,S,SAF,SS,WR
0,20170907000118,24.662644,72,197,46.65,14.503333,345.47,10.84,1,0.855705,4.468811,4.55,0.76,0.51,9.902020,146.951875,81.211875,2.176268,3.663441,5.799078,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20170907000118,30.061685,72,206,50.68,17.913333,282.63,344.31,1,-0.492187,1.752185,1.82,2.43,0.16,10.622476,117.409349,51.669349,5.836525,5.890306,6.356886,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,20170907000118,28.692760,72,212,46.09,18.493333,351.99,357.18,1,-0.083145,1.687953,1.69,1.13,0.40,6.480872,131.684932,65.944932,3.834835,1.847647,2.866234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,20170907000118,28.629790,75,270,46.00,20.133333,273.01,22.73,1,0.471395,1.125250,1.22,0.59,0.31,5.448982,119.340570,53.600570,4.466378,0.780064,1.449966,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20170907000118,28.457305,75,288,45.33,20.693333,297.61,18.70,1,0.134657,0.397828,0.42,1.35,0.01,4.593310,117.346055,51.606055,10.936453,0.580517,0.763217,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,20170907000118,23.184204,78,308,45.85,24.433333,252.58,94.14,1,0.718121,-0.051980,0.72,0.73,0.01,4.880256,70.488329,4.748329,6.778134,0.483011,0.608112,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,20170907000118,34.795430,75,245,48.54,25.633333,269.77,285.64,1,-0.404449,0.113229,0.42,0.54,0.02,7.820038,68.783638,3.043638,18.619139,3.213612,3.427944,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,20170907000118,27.512746,74,208,63.37,26.433333,254.70,235.31,1,-0.213783,-0.147975,0.26,1.86,0.28,22.415872,80.680549,14.940549,86.214891,18.018529,18.086760,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,20170907000118,26.475101,74,252,44.94,29.333333,218.34,275.01,1,-1.006141,0.088203,1.01,0.32,0.18,7.500467,29.470106,36.269894,7.426205,1.400000,4.495698,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,20170907000118,26.431295,72,193,46.63,34.603333,238.52,90.04,1,1.240000,-0.000866,1.24,0.74,0.13,12.968593,24.509770,41.230230,10.458543,1.407302,2.715032,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [95]:
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 10)].reset_index(drop = True) for k in range(10)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 330)

In [96]:
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 11)].reset_index(drop = True) for k in range(11)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape

(23171, 418)

In [97]:
X = np.hstack([rusher.drop('PlayId',axis = 1).values,game.drop('PlayId',axis = 1).values,offender_players,defender_players])
Y = np.zeros((y.Yards.shape[0], 199))
for idx, target in enumerate(list(y.Yards)):
    Y[idx][99 + target] = 1

In [98]:
print(np.any(np.isnan(X)))
print(np.all(np.isfinite(X)))

False
True


In [99]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [100]:
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
#         X_train, y_train = self.data[0][0], self.data[0][1]
#         y_pred = self.model.predict(X_train)
#         y_true = np.clip(np.cumsum(y_train, axis=1), 0, 1)
#         y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
#         tr_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_train.shape[0])
#         tr_s = np.round(tr_s, 6)
        logs['tr_CRPS'] = 0

        X_valid, y_valid = self.data[1][0], self.data[1][1]

        y_pred = self.model.predict(X_valid)
        y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
        val_s = np.round(val_s, 6)
        logs['val_CRPS'] = val_s
        print('tr CRPS', 'Grr', 'val CRPS', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [103]:
def get_model():
    model = Sequential()

    model.add(Dropout(0.1))
    model.add(GaussianNoise(0.25))
    model.add(Dense(199, activation=None))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))


    model.add(Dropout(0.1))
    model.add(GaussianNoise(0.1))
    model.add(Dense(199, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=[])
   
    return model

In [104]:
models = []


rkf = GroupKFold(6)
for fold_id, (tr_idx, vl_idx) in enumerate(rkf.split(rusher['PlayId'],rusher['PlayId'],rusher['PlayId'])):
    print(fold_id)

    x_tr, y_tr = X[tr_idx], Y[tr_idx]
    x_vl, y_vl = X[vl_idx], Y[vl_idx]    

    model= get_model()
    es = EarlyStopping(monitor='val_CRPS', 
                   mode='min',
                   restore_best_weights=True, 
                   verbose=1, 
                   patience=5)
    es.set_model(model)
    metric = Metric(model, [es], [(x_tr, y_tr), (x_vl, y_vl)])



    model.fit(x_tr, y_tr, callbacks=[metric], epochs=250, batch_size=100, validation_data=[x_vl, y_vl])
    models.append(model)

0
Train on 19309 samples, validate on 3862 samples
Epoch 1/250
19309/19309 [==============================] - ETA: 49s - loss: 5.45 - ETA: 9s - loss: 5.5047 - ETA: 5s - loss: 5.463 - ETA: 4s - loss: 5.444 - ETA: 3s - loss: 5.421 - ETA: 3s - loss: 5.395 - ETA: 2s - loss: 5.369 - ETA: 2s - loss: 5.338 - ETA: 2s - loss: 5.306 - ETA: 2s - loss: 5.282 - ETA: 1s - loss: 5.247 - ETA: 1s - loss: 5.223 - ETA: 1s - loss: 5.195 - ETA: 1s - loss: 5.158 - ETA: 1s - loss: 5.130 - ETA: 1s - loss: 5.105 - ETA: 1s - loss: 5.075 - ETA: 1s - loss: 5.048 - ETA: 1s - loss: 5.013 - ETA: 0s - loss: 4.983 - ETA: 0s - loss: 4.960 - ETA: 0s - loss: 4.926 - ETA: 0s - loss: 4.890 - ETA: 0s - loss: 4.858 - ETA: 0s - loss: 4.827 - ETA: 0s - loss: 4.798 - ETA: 0s - loss: 4.760 - ETA: 0s - loss: 4.725 - ETA: 0s - loss: 4.695 - ETA: 0s - loss: 4.657 - ETA: 0s - loss: 4.619 - ETA: 0s - loss: 4.588 - ETA: 0s - loss: 4.551 - ETA: 0s - loss: 4.517 - 2s 118us/step - loss: 4.5055 - val_loss: 3.2424
tr CRPS Grr val CRPS 0.01

19309/19309 [==============================] - ETA: 1s - loss: 2.471 - ETA: 1s - loss: 2.493 - ETA: 1s - loss: 2.492 - ETA: 1s - loss: 2.497 - ETA: 1s - loss: 2.492 - ETA: 1s - loss: 2.490 - ETA: 1s - loss: 2.487 - ETA: 1s - loss: 2.494 - ETA: 1s - loss: 2.496 - ETA: 1s - loss: 2.504 - ETA: 1s - loss: 2.504 - ETA: 1s - loss: 2.509 - ETA: 1s - loss: 2.508 - ETA: 1s - loss: 2.511 - ETA: 1s - loss: 2.513 - ETA: 0s - loss: 2.511 - ETA: 0s - loss: 2.518 - ETA: 0s - loss: 2.517 - ETA: 0s - loss: 2.514 - ETA: 0s - loss: 2.519 - ETA: 0s - loss: 2.520 - ETA: 0s - loss: 2.522 - ETA: 0s - loss: 2.524 - ETA: 0s - loss: 2.528 - ETA: 0s - loss: 2.529 - ETA: 0s - loss: 2.531 - ETA: 0s - loss: 2.530 - ETA: 0s - loss: 2.531 - ETA: 0s - loss: 2.533 - ETA: 0s - loss: 2.534 - ETA: 0s - loss: 2.537 - ETA: 0s - loss: 2.540 - ETA: 0s - loss: 2.539 - 2s 100us/step - loss: 2.5401 - val_loss: 2.8426
tr CRPS Grr val CRPS 0.012834
Epoch 10/250
19309/19309 [==============================] - ETA: 1s - loss: 2.421 -

19309/19309 [==============================] - ETA: 1s - loss: 2.647 - ETA: 1s - loss: 2.608 - ETA: 1s - loss: 2.637 - ETA: 1s - loss: 2.632 - ETA: 1s - loss: 2.644 - ETA: 1s - loss: 2.648 - ETA: 1s - loss: 2.648 - ETA: 1s - loss: 2.654 - ETA: 1s - loss: 2.651 - ETA: 1s - loss: 2.651 - ETA: 1s - loss: 2.657 - ETA: 1s - loss: 2.669 - ETA: 1s - loss: 2.670 - ETA: 1s - loss: 2.674 - ETA: 1s - loss: 2.674 - ETA: 0s - loss: 2.680 - ETA: 0s - loss: 2.681 - ETA: 0s - loss: 2.681 - ETA: 0s - loss: 2.682 - ETA: 0s - loss: 2.680 - ETA: 0s - loss: 2.679 - ETA: 0s - loss: 2.682 - ETA: 0s - loss: 2.681 - ETA: 0s - loss: 2.682 - ETA: 0s - loss: 2.683 - ETA: 0s - loss: 2.683 - ETA: 0s - loss: 2.681 - ETA: 0s - loss: 2.683 - ETA: 0s - loss: 2.683 - ETA: 0s - loss: 2.688 - ETA: 0s - loss: 2.686 - ETA: 0s - loss: 2.687 - ETA: 0s - loss: 2.688 - ETA: 0s - loss: 2.690 - 2s 101us/step - loss: 2.6900 - val_loss: 2.7954
tr CRPS Grr val CRPS 0.013209
Epoch 5/250
19309/19309 [==============================] - 

19309/19309 [==============================] - ETA: 1s - loss: 2.828 - ETA: 1s - loss: 2.760 - ETA: 1s - loss: 2.704 - ETA: 1s - loss: 2.729 - ETA: 1s - loss: 2.725 - ETA: 1s - loss: 2.725 - ETA: 1s - loss: 2.735 - ETA: 1s - loss: 2.736 - ETA: 1s - loss: 2.727 - ETA: 1s - loss: 2.726 - ETA: 1s - loss: 2.733 - ETA: 1s - loss: 2.733 - ETA: 1s - loss: 2.733 - ETA: 1s - loss: 2.729 - ETA: 0s - loss: 2.731 - ETA: 0s - loss: 2.733 - ETA: 0s - loss: 2.733 - ETA: 0s - loss: 2.731 - ETA: 0s - loss: 2.733 - ETA: 0s - loss: 2.732 - ETA: 0s - loss: 2.731 - ETA: 0s - loss: 2.734 - ETA: 0s - loss: 2.735 - ETA: 0s - loss: 2.738 - ETA: 0s - loss: 2.738 - ETA: 0s - loss: 2.737 - ETA: 0s - loss: 2.739 - ETA: 0s - loss: 2.740 - ETA: 0s - loss: 2.742 - ETA: 0s - loss: 2.740 - ETA: 0s - loss: 2.742 - ETA: 0s - loss: 2.742 - ETA: 0s - loss: 2.742 - 2s 98us/step - loss: 2.7426 - val_loss: 2.8193
tr CRPS Grr val CRPS 0.013078
Epoch 4/250
19309/19309 [==============================] - ETA: 2s - loss: 2.499 - E

19309/19309 [==============================] - ETA: 2s - loss: 2.313 - ETA: 1s - loss: 2.408 - ETA: 1s - loss: 2.428 - ETA: 1s - loss: 2.423 - ETA: 1s - loss: 2.416 - ETA: 1s - loss: 2.434 - ETA: 1s - loss: 2.431 - ETA: 1s - loss: 2.439 - ETA: 1s - loss: 2.456 - ETA: 1s - loss: 2.463 - ETA: 1s - loss: 2.462 - ETA: 1s - loss: 2.465 - ETA: 1s - loss: 2.471 - ETA: 1s - loss: 2.472 - ETA: 1s - loss: 2.471 - ETA: 0s - loss: 2.473 - ETA: 0s - loss: 2.474 - ETA: 0s - loss: 2.472 - ETA: 0s - loss: 2.474 - ETA: 0s - loss: 2.476 - ETA: 0s - loss: 2.479 - ETA: 0s - loss: 2.480 - ETA: 0s - loss: 2.484 - ETA: 0s - loss: 2.487 - ETA: 0s - loss: 2.489 - ETA: 0s - loss: 2.491 - ETA: 0s - loss: 2.491 - ETA: 0s - loss: 2.492 - ETA: 0s - loss: 2.493 - ETA: 0s - loss: 2.496 - ETA: 0s - loss: 2.496 - ETA: 0s - loss: 2.498 - ETA: 0s - loss: 2.498 - 2s 98us/step - loss: 2.4996 - val_loss: 2.8711
tr CRPS Grr val CRPS 0.013028
Epoch 12/250
19309/19309 [==============================] - ETA: 1s - loss: 2.499 - 

19309/19309 [==============================] - ETA: 2s - loss: 2.809 - ETA: 1s - loss: 2.624 - ETA: 1s - loss: 2.594 - ETA: 1s - loss: 2.573 - ETA: 1s - loss: 2.573 - ETA: 1s - loss: 2.572 - ETA: 1s - loss: 2.575 - ETA: 1s - loss: 2.574 - ETA: 1s - loss: 2.574 - ETA: 1s - loss: 2.568 - ETA: 1s - loss: 2.570 - ETA: 1s - loss: 2.570 - ETA: 1s - loss: 2.571 - ETA: 1s - loss: 2.570 - ETA: 1s - loss: 2.565 - ETA: 0s - loss: 2.564 - ETA: 0s - loss: 2.565 - ETA: 0s - loss: 2.570 - ETA: 0s - loss: 2.569 - ETA: 0s - loss: 2.571 - ETA: 0s - loss: 2.576 - ETA: 0s - loss: 2.580 - ETA: 0s - loss: 2.582 - ETA: 0s - loss: 2.580 - ETA: 0s - loss: 2.580 - ETA: 0s - loss: 2.580 - ETA: 0s - loss: 2.581 - ETA: 0s - loss: 2.580 - ETA: 0s - loss: 2.583 - ETA: 0s - loss: 2.583 - ETA: 0s - loss: 2.584 - ETA: 0s - loss: 2.583 - ETA: 0s - loss: 2.586 - 2s 102us/step - loss: 2.5867 - val_loss: 2.8611
tr CRPS Grr val CRPS 0.013572
Epoch 8/250
19309/19309 [==============================] - ETA: 2s - loss: 2.624 - 

19309/19309 [==============================] - ETA: 2s - loss: 2.588 - ETA: 2s - loss: 2.498 - ETA: 1s - loss: 2.540 - ETA: 1s - loss: 2.538 - ETA: 1s - loss: 2.551 - ETA: 1s - loss: 2.581 - ETA: 1s - loss: 2.582 - ETA: 1s - loss: 2.582 - ETA: 1s - loss: 2.585 - ETA: 1s - loss: 2.597 - ETA: 1s - loss: 2.601 - ETA: 1s - loss: 2.602 - ETA: 1s - loss: 2.604 - ETA: 1s - loss: 2.603 - ETA: 1s - loss: 2.603 - ETA: 1s - loss: 2.607 - ETA: 1s - loss: 2.611 - ETA: 0s - loss: 2.617 - ETA: 0s - loss: 2.618 - ETA: 0s - loss: 2.623 - ETA: 0s - loss: 2.622 - ETA: 0s - loss: 2.624 - ETA: 0s - loss: 2.626 - ETA: 0s - loss: 2.631 - ETA: 0s - loss: 2.632 - ETA: 0s - loss: 2.634 - ETA: 0s - loss: 2.640 - ETA: 0s - loss: 2.642 - ETA: 0s - loss: 2.644 - ETA: 0s - loss: 2.645 - ETA: 0s - loss: 2.645 - ETA: 0s - loss: 2.647 - ETA: 0s - loss: 2.647 - ETA: 0s - loss: 2.646 - 2s 104us/step - loss: 2.6469 - val_loss: 2.8472
tr CRPS Grr val CRPS 0.013597
Epoch 6/250
19309/19309 [==============================] - 

19310/19310 [==============================] - ETA: 46s - loss: 5.55 - ETA: 9s - loss: 5.5685 - ETA: 5s - loss: 5.514 - ETA: 4s - loss: 5.505 - ETA: 3s - loss: 5.487 - ETA: 3s - loss: 5.453 - ETA: 2s - loss: 5.421 - ETA: 2s - loss: 5.398 - ETA: 2s - loss: 5.357 - ETA: 2s - loss: 5.326 - ETA: 2s - loss: 5.303 - ETA: 1s - loss: 5.272 - ETA: 1s - loss: 5.241 - ETA: 1s - loss: 5.209 - ETA: 1s - loss: 5.171 - ETA: 1s - loss: 5.143 - ETA: 1s - loss: 5.112 - ETA: 1s - loss: 5.076 - ETA: 1s - loss: 5.044 - ETA: 1s - loss: 5.017 - ETA: 0s - loss: 4.981 - ETA: 0s - loss: 4.952 - ETA: 0s - loss: 4.917 - ETA: 0s - loss: 4.884 - ETA: 0s - loss: 4.847 - ETA: 0s - loss: 4.812 - ETA: 0s - loss: 4.776 - ETA: 0s - loss: 4.742 - ETA: 0s - loss: 4.703 - ETA: 0s - loss: 4.661 - ETA: 0s - loss: 4.626 - ETA: 0s - loss: 4.591 - ETA: 0s - loss: 4.556 - ETA: 0s - loss: 4.521 - 2s 116us/step - loss: 4.5141 - val_loss: 3.2301
tr CRPS Grr val CRPS 0.019638
Epoch 2/250
19310/19310 [==============================] -

19310/19310 [==============================] - ETA: 1s - loss: 2.445 - ETA: 1s - loss: 2.454 - ETA: 1s - loss: 2.446 - ETA: 1s - loss: 2.446 - ETA: 1s - loss: 2.473 - ETA: 1s - loss: 2.476 - ETA: 1s - loss: 2.481 - ETA: 1s - loss: 2.487 - ETA: 1s - loss: 2.488 - ETA: 1s - loss: 2.490 - ETA: 1s - loss: 2.490 - ETA: 1s - loss: 2.492 - ETA: 1s - loss: 2.497 - ETA: 1s - loss: 2.501 - ETA: 0s - loss: 2.501 - ETA: 0s - loss: 2.501 - ETA: 0s - loss: 2.503 - ETA: 0s - loss: 2.508 - ETA: 0s - loss: 2.510 - ETA: 0s - loss: 2.513 - ETA: 0s - loss: 2.515 - ETA: 0s - loss: 2.518 - ETA: 0s - loss: 2.517 - ETA: 0s - loss: 2.520 - ETA: 0s - loss: 2.522 - ETA: 0s - loss: 2.524 - ETA: 0s - loss: 2.526 - ETA: 0s - loss: 2.528 - ETA: 0s - loss: 2.530 - ETA: 0s - loss: 2.531 - ETA: 0s - loss: 2.533 - ETA: 0s - loss: 2.536 - ETA: 0s - loss: 2.538 - 2s 95us/step - loss: 2.5380 - val_loss: 2.8577
tr CRPS Grr val CRPS 0.013516
Epoch 10/250
19310/19310 [==============================] - ETA: 2s - loss: 2.411 - 

In [86]:
class GP:
    def __init__(self):
        self.classes = 20
        self.class_names = [ 'class_0',
                             'class_1',
                             'class_2',
                             'class_3',
                             'class_4',
                             'class_5',
                             'class_6',
                             'class_7',
                             'class_8',
                             'class_9',
                             'class_10',
                             'class_11',
                             'class_12',
                             'class_13',
                            'class_14',
                            'class_15',
                            'class_16',
                            'class_17',
                            'class_18',
                            'class_19',
                           ]


    def GrabPredictions(self, data):
        oof_preds = np.zeros((len(data), len(self.class_names)))
        oof_preds[:,0] = self.GP_class_0(data)
        oof_preds[:,1] = self.GP_class_1(data)
        oof_preds[:,2] = self.GP_class_2(data)
        oof_preds[:,3] = self.GP_class_3(data)
        oof_preds[:,4] = self.GP_class_4(data)
        oof_preds[:,5] = self.GP_class_5(data)
        oof_preds[:,6] = self.GP_class_6(data)
        oof_preds[:,7] = self.GP_class_7(data)
        oof_preds[:,8] = self.GP_class_8(data)
        oof_preds[:,9] = self.GP_class_9(data)
        oof_preds[:,10] = self.GP_class_10(data)
        oof_preds[:,11] = self.GP_class_11(data)
        oof_preds[:,12] = self.GP_class_12(data)
        oof_preds[:,13] = self.GP_class_13(data)
        oof_preds[:,14] = self.GP_class_14(data)
        oof_preds[:,15] = self.GP_class_15(data)
        oof_preds[:,16] = self.GP_class_16(data)
        oof_preds[:,17] = self.GP_class_17(data)
        oof_preds[:,18] = self.GP_class_18(data)
        oof_preds[:,19] = self.GP_class_19(data)
        oof_df = pd.DataFrame(np.exp(oof_preds), columns=self.class_names)
        oof_df =oof_df.div(oof_df.sum(axis=1), axis=0)
        
        return oof_df.values


    def GP_class_0(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) - (((data[:,0]) + (data[:,0]))))) + (data[:,0]))) + (((((data[:,0]) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((((data[:,2]) - (data[:,7]))) * 2.0)) + (data[:,7]))) * 2.0)) + (data[:,0]))) + (((((data[:,2]) + (data[:,0]))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((((((((data[:,3]) - (data[:,7]))) - (((data[:,0]) / 2.0)))) + (data[:,0]))) - (data[:,7]))) + (data[:,2]))) + (((data[:,7]) + (data[:,2]))))) + (data[:,7]))) - (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((((((data[:,0]) - (data[:,7]))) - (data[:,14]))) + (((data[:,17]) - (data[:,14]))))) + (((((((data[:,0]) - ((1.0)))) - (data[:,0]))) + (data[:,20]))))) +
                0.250000*np.tanh(((((data[:,18]) + (((((((data[:,0]) - (data[:,7]))) + (data[:,2]))) + (data[:,0]))))) - (((data[:,7]) + (data[:,2]))))) +
                0.250000*np.tanh((((((((data[:,8]) + ((((((((data[:,13]) + (data[:,3]))) + (data[:,0]))/2.0)) + ((((((((data[:,0]) + (((((data[:,0]) - (data[:,0]))) / 2.0)))) + (data[:,17]))/2.0)) + (data[:,11]))))))/2.0)) + (data[:,0]))) - (data[:,13]))) +
                0.250000*np.tanh(((((data[:,0]) * 2.0)) + (((((((((data[:,2]) + (data[:,2]))) * 2.0)) - (((data[:,14]) - (((np.tanh((data[:,2]))) * 2.0)))))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((data[:,17]) - (((data[:,17]) * (((((data[:,17]) * (data[:,17]))) * 2.0)))))) + (((((((data[:,17]) * 2.0)) * 2.0)) * 2.0)))/2.0)) / 2.0)) + (((data[:,17]) - ((7.76236486434936523)))))/2.0)) + (((((((data[:,17]) * (data[:,17]))) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((data[:,17]) + (data[:,17]))/2.0)) + ((((data[:,17]) + ((((data[:,17]) + (data[:,17]))/2.0)))/2.0)))/2.0)) * (data[:,17]))) - (data[:,13]))) +
                0.250000*np.tanh((((((data[:,19]) + (((((((((data[:,19]) * 2.0)) + (data[:,19]))) - ((1.07645297050476074)))) + (((data[:,22]) + (data[:,22]))))))/2.0)) + (((data[:,22]) + (data[:,4]))))))
    
    def GP_class_1(self,data):
        return(0.250000*np.tanh(((((((((((((data[:,3]) + (((np.tanh((data[:,2]))) - (data[:,14]))))) - ((10.0)))) - ((4.88989353179931641)))) - ((13.92175483703613281)))) / 2.0)) - (data[:,6]))) +
                0.250000*np.tanh(data[:,3]) +
                0.250000*np.tanh(((data[:,2]) + (((((np.tanh((((data[:,14]) * 2.0)))) - (((((((data[:,7]) * 2.0)) - (((((data[:,0]) - (data[:,14]))) - (data[:,7]))))) - (data[:,0]))))) + (data[:,0]))))) +
                0.250000*np.tanh(((((((data[:,0]) - ((-1.0*((data[:,14])))))) + ((((((((((data[:,0]) - (data[:,7]))) - ((((data[:,14]) + (data[:,14]))/2.0)))) + (((data[:,2]) - (((data[:,0]) * (data[:,14]))))))/2.0)) - (data[:,14]))))) - (data[:,7]))) +
                0.250000*np.tanh(((((data[:,0]) / 2.0)) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,19]) - (data[:,19]))) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((np.tanh((data[:,5]))) - ((((1.69341480731964111)) + ((((((np.tanh((data[:,14]))) - (data[:,14]))) + (data[:,14]))/2.0)))))))) - (data[:,2]))) - (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh((((((data[:,22]) + (((data[:,0]) + ((((((((data[:,22]) / 2.0)) + ((-1.0*((data[:,13])))))/2.0)) + (data[:,19]))))))/2.0)) - (data[:,13]))) +
                0.250000*np.tanh(((((((data[:,17]) / 2.0)) * (data[:,17]))) * (((data[:,17]) + (((data[:,17]) * (((((((((data[:,17]) + (data[:,17]))) * (data[:,17]))) * (data[:,17]))) + (((np.tanh((data[:,17]))) / 2.0)))))))))) +
                0.250000*np.tanh((((((data[:,14]) * (((data[:,13]) + (data[:,3]))))) + (((data[:,14]) * (((data[:,14]) - ((-1.0*((data[:,3])))))))))/2.0)))
    
    def GP_class_2(self,data):
        return(0.250000*np.tanh(((data[:,0]) + (np.tanh((((((((((data[:,2]) + (((data[:,2]) / 2.0)))/2.0)) + (data[:,0]))/2.0)) * (data[:,7]))))))) +
                0.250000*np.tanh((((data[:,22]) + (data[:,22]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + ((-1.0*((((((4.73206138610839844)) + (((((((((((np.tanh((data[:,7]))) + (data[:,1]))/2.0)) - (np.tanh((((((data[:,0]) - (data[:,20]))) - (data[:,14]))))))) * ((7.0)))) + (data[:,14]))/2.0)))/2.0))))))/2.0)) - (data[:,14]))) +
                0.250000*np.tanh(((((data[:,1]) + ((4.15603733062744141)))) / 2.0)) +
                0.250000*np.tanh(((((((((((((data[:,0]) - (data[:,7]))) - (data[:,7]))) + ((((data[:,0]) + (data[:,0]))/2.0)))) * 2.0)) + (data[:,0]))) * 2.0)) +
                0.250000*np.tanh(((((data[:,11]) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((((((((((((((data[:,2]) / 2.0)) + (((data[:,2]) / 2.0)))/2.0)) / 2.0)) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,0]) / 2.0))))) * (((data[:,0]) / 2.0)))))/2.0)) + (data[:,0]))/2.0)) + (np.tanh((data[:,0]))))/2.0)) / 2.0)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + (((((((data[:,14]) * (data[:,14]))) + (data[:,14]))) - (data[:,0]))))))) - (data[:,14]))) +
                0.250000*np.tanh((((((np.tanh((np.tanh(((0.0)))))) * (data[:,15]))) + (data[:,20]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + ((((data[:,13]) + ((((data[:,14]) + ((((((data[:,14]) + ((((data[:,14]) + (((data[:,13]) - ((((-1.0*(((((data[:,14]) + (data[:,14]))/2.0))))) * 2.0)))))/2.0)))) + (data[:,14]))/2.0)))/2.0)))/2.0)))/2.0)))))
    
    def GP_class_3(self,data):
        return(0.250000*np.tanh(((((((((8.0)) + (((((((5.33416414260864258)) + ((9.0)))/2.0)) + ((((8.0)) * 2.0)))))) * 2.0)) + ((5.33416414260864258)))/2.0)) +
                0.250000*np.tanh((((9.48088836669921875)) + (((((10.56953334808349609)) + ((((4.48959350585937500)) + (np.tanh(((((3.0)) + ((9.0)))))))))/2.0)))) +
                0.250000*np.tanh((((((((((data[:,22]) / 2.0)) + (data[:,22]))/2.0)) * 2.0)) / 2.0)) +
                0.250000*np.tanh((10.44883441925048828)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + ((-1.0*((data[:,9])))))))) + (((((data[:,0]) - (data[:,7]))) - (data[:,9]))))) +
                0.250000*np.tanh(((np.tanh((data[:,6]))) - (data[:,21]))) +
                0.250000*np.tanh(((data[:,0]) - (((((data[:,14]) + (((((data[:,14]) + (((data[:,14]) + (((data[:,14]) - (data[:,0]))))))) - (((((data[:,0]) - (data[:,14]))) + (data[:,14]))))))) - (((((data[:,14]) * (data[:,14]))) + (data[:,18]))))))) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + (data[:,14]))/2.0)))) +
                0.250000*np.tanh((((((data[:,8]) * (((data[:,8]) / 2.0)))) + ((((((data[:,9]) * (data[:,8]))) + (data[:,8]))/2.0)))/2.0)) +
                0.250000*np.tanh((((data[:,2]) + (((((((data[:,2]) + (data[:,20]))/2.0)) + ((((((((data[:,2]) + (data[:,8]))/2.0)) / 2.0)) - (data[:,8]))))/2.0)))/2.0)))
    
    def GP_class_4(self,data):
        return(0.250000*np.tanh((((12.98819637298583984)) / 2.0)) +
                0.250000*np.tanh(((((4.75780344009399414)) + (((((((8.66014671325683594)) + ((4.18107128143310547)))/2.0)) * ((8.97841739654541016)))))/2.0)) +
                0.250000*np.tanh((((9.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((10.0)) + (np.tanh((((((6.0)) + ((((13.80149555206298828)) + ((7.0)))))/2.0)))))) + ((10.0)))) + (((((((data[:,0]) + ((8.0)))) / 2.0)) / 2.0)))/2.0)) + ((((-1.0*((((data[:,2]) * (data[:,9])))))) * 2.0)))) +
                0.250000*np.tanh(((((data[:,0]) + (((((((data[:,22]) + (data[:,22]))/2.0)) + (data[:,22]))/2.0)))) + ((((((data[:,22]) + ((((((data[:,22]) * 2.0)) + (data[:,22]))/2.0)))) + (data[:,18]))/2.0)))) +
                0.250000*np.tanh(((((((((((((np.tanh((data[:,18]))) + ((((data[:,1]) + (data[:,18]))/2.0)))/2.0)) * (data[:,18]))) * (data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) - (((((((np.tanh((data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) + (data[:,17]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((np.tanh((data[:,6]))) - (((np.tanh((data[:,6]))) - (data[:,6]))))) + (np.tanh((data[:,6]))))/2.0)))) - ((((data[:,10]) + ((((data[:,2]) + (((np.tanh((data[:,6]))) - (((np.tanh((np.tanh((data[:,6]))))) - (data[:,10]))))))/2.0)))/2.0)))) +
                0.250000*np.tanh((((((((((((data[:,10]) + (data[:,4]))) + (((data[:,10]) + (((data[:,0]) * (data[:,10]))))))/2.0)) + (((data[:,4]) * (((data[:,10]) + (data[:,4]))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,11]) + (((((data[:,18]) / 2.0)) * (((data[:,11]) * (data[:,18]))))))/2.0)) * (((data[:,18]) * (((((((((data[:,18]) * (((((((data[:,11]) / 2.0)) / 2.0)) * (data[:,18]))))) / 2.0)) / 2.0)) * ((-1.0*((data[:,21])))))))))) +
                0.250000*np.tanh((((-1.0*((((((((((((data[:,2]) - (data[:,2]))) * (np.tanh((data[:,2]))))) + (np.tanh((((data[:,14]) * 2.0)))))/2.0)) + (data[:,2]))/2.0))))) - (data[:,2]))))
    
    def GP_class_5(self,data):
        return(0.250000*np.tanh(((((((((((3.46574378013610840)) + ((((np.tanh(((3.46574378013610840)))) + ((8.46705245971679688)))/2.0)))/2.0)) * 2.0)) + ((((3.46574378013610840)) + ((4.0)))))) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((10.55856513977050781)) / 2.0)) +
                0.250000*np.tanh((((3.76695251464843750)) / 2.0)) +
                0.250000*np.tanh((((((6.0)) / 2.0)) + ((((8.57984828948974609)) * 2.0)))) +
                0.250000*np.tanh((((((((((((3.42168045043945312)) + (data[:,7]))) + (np.tanh((data[:,7]))))) + (np.tanh((np.tanh((np.tanh(((3.42168045043945312)))))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,6]) - (data[:,9]))) + (data[:,7]))/2.0)) * 2.0)) + ((((((data[:,7]) + (data[:,9]))/2.0)) - (data[:,9]))))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,22]) + (data[:,22]))) + (data[:,11]))/2.0)) + (data[:,22]))/2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((0.76044219732284546)) / 2.0)) + ((0.76044219732284546)))/2.0)) + (np.tanh((np.tanh((((((((0.76043862104415894)) / 2.0)) + ((1.0)))/2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((data[:,12]) + (np.tanh((data[:,9]))))/2.0)) + (((((((((((data[:,12]) + (((((0.0)) + (np.tanh((((((((data[:,7]) + (data[:,6]))/2.0)) + (data[:,7]))/2.0)))))/2.0)))/2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,20]) / 2.0)) / 2.0)) * (((np.tanh((data[:,20]))) * (((data[:,20]) / 2.0)))))) * ((((((data[:,20]) / 2.0)) + (((data[:,20]) / 2.0)))/2.0)))))
    
    def GP_class_6(self,data):
        return(0.250000*np.tanh(((((((11.68076229095458984)) + (((((11.68075847625732422)) + ((((11.47270107269287109)) + (((((11.47269725799560547)) + ((11.68076229095458984)))/2.0)))))/2.0)))/2.0)) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((8.0)) + ((((10.0)) + ((6.59042119979858398)))))) +
                0.250000*np.tanh(((((((((((13.33760547637939453)) + (((((3.86388397216796875)) + (((((((8.18321037292480469)) + (data[:,4]))) + (((((((5.95386552810668945)) + ((12.93883609771728516)))) + (((((10.73907089233398438)) + ((4.0)))/2.0)))/2.0)))/2.0)))/2.0)))/2.0)) + (np.tanh(((3.0)))))) - ((2.0)))) + (((((10.61559963226318359)) + (data[:,1]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((((((7.13513946533203125)) + ((12.48778533935546875)))/2.0)) + (((((((7.13513565063476562)) - ((8.12031841278076172)))) + (((((((((7.13513565063476562)) + ((((10.69830417633056641)) + ((10.69830799102783203)))))) + ((10.69830799102783203)))) + ((7.83078956604003906)))/2.0)))/2.0)))/2.0)) * 2.0)))) + ((3.0)))) +
                0.250000*np.tanh(((((((((((data[:,7]) - (data[:,0]))) * 2.0)) - (data[:,7]))) + (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,7]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((1.0)) + (((((((data[:,2]) + (((((((1.0)) * (data[:,2]))) + (data[:,14]))/2.0)))/2.0)) + ((1.0)))/2.0)))) + ((((1.0)) - (((data[:,2]) * (data[:,14]))))))/2.0)) - (data[:,2]))) * 2.0)) + (data[:,14]))/2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((((data[:,19]) * ((0.0)))) + ((0.0)))/2.0)))))) / 2.0)) +
                0.250000*np.tanh((((((((0.0)) / 2.0)) / 2.0)) * (((((((((data[:,15]) / 2.0)) * ((0.0)))) / 2.0)) * ((((-1.0*(((0.0))))) / 2.0)))))) +
                0.250000*np.tanh(((((data[:,15]) * (((((((((((((np.tanh(((((0.09032609313726425)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))) / 2.0)))
    
    def GP_class_7(self,data):
        return(0.250000*np.tanh((10.27210903167724609)) +
                0.250000*np.tanh((((((((((10.0)) + (((((((((8.26972770690917969)) + ((11.06334972381591797)))/2.0)) * 2.0)) / 2.0)))) + ((((((5.0)) + ((((5.0)) + ((7.92727756500244141)))))) * 2.0)))) * ((7.92728137969970703)))) + ((10.0)))) +
                0.250000*np.tanh((8.42519950866699219)) +
                0.250000*np.tanh(((((((data[:,14]) + (data[:,14]))) + ((((((1.59392631053924561)) + (data[:,14]))) + (data[:,14]))))) + (np.tanh((((((data[:,14]) + (((((((3.0)) + (((data[:,14]) * 2.0)))/2.0)) + ((1.59391915798187256)))))) + ((1.59392631053924561)))))))) +
                0.250000*np.tanh((((data[:,7]) + (((data[:,0]) + (((((((data[:,7]) + (data[:,4]))/2.0)) + (((data[:,7]) * 2.0)))/2.0)))))/2.0)) +
                0.250000*np.tanh((((((((((((((data[:,9]) - (((data[:,0]) - (data[:,0]))))) - (((data[:,0]) / 2.0)))) / 2.0)) - (((data[:,21]) - (data[:,7]))))) + (data[:,21]))/2.0)) - (data[:,0]))) +
                0.250000*np.tanh((((((((((2.0)) + ((((((data[:,21]) + ((((data[:,13]) + (((((((((((data[:,13]) * 2.0)) + (data[:,21]))/2.0)) + (data[:,13]))/2.0)) * 2.0)))/2.0)))/2.0)) - (np.tanh(((((0.0)) - (data[:,21]))))))))) + ((1.0)))/2.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((-1.0*(((((((data[:,21]) / 2.0)) + (data[:,8]))/2.0))))) / 2.0)) / 2.0)) / 2.0)) + (data[:,9]))/2.0)) / 2.0)) + ((((((((((((data[:,7]) / 2.0)) / 2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((np.tanh(((1.0)))) / 2.0)) / 2.0)) / 2.0)))
    
    def GP_class_8(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) + (((((6.71804094314575195)) + (data[:,15]))/2.0)))) + ((((((11.56385326385498047)) * 2.0)) * ((((10.48986148834228516)) + ((11.56385326385498047)))))))) / 2.0)) +
                0.250000*np.tanh((((((((((data[:,14]) / 2.0)) + ((2.83755254745483398)))/2.0)) + ((((((data[:,12]) + (data[:,14]))/2.0)) + (data[:,14]))))) + ((((data[:,9]) + (((data[:,12]) + (data[:,14]))))/2.0)))) +
                0.250000*np.tanh(((((((((data[:,6]) + (((((((3.50821948051452637)) + ((((data[:,21]) + ((11.92932796478271484)))/2.0)))) + ((1.0)))/2.0)))) + (data[:,10]))/2.0)) + (data[:,21]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (np.tanh((np.tanh(((-1.0*((((data[:,22]) - (data[:,13])))))))))))) +
                0.250000*np.tanh(((((1.0)) + ((((data[:,7]) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,21]) / 2.0)) / 2.0)) * (((((((np.tanh((((np.tanh((data[:,13]))) / 2.0)))) / 2.0)) + (((((((np.tanh(((((data[:,21]) + (data[:,21]))/2.0)))) / 2.0)) - (data[:,12]))) / 2.0)))) / 2.0)))) - (data[:,12]))) +
                0.250000*np.tanh((((data[:,7]) + (np.tanh(((((data[:,12]) + (((data[:,7]) * ((((data[:,13]) + (data[:,13]))/2.0)))))/2.0)))))/2.0)) +
                0.250000*np.tanh(((((((data[:,13]) / 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((np.tanh((data[:,9]))) + (data[:,13]))/2.0)) + ((((((((data[:,21]) / 2.0)) * 2.0)) + ((((3.94983267784118652)) / 2.0)))/2.0)))/2.0)) + ((((np.tanh(((3.94983267784118652)))) + (data[:,9]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((((((((data[:,14]) * (((((((((data[:,19]) / 2.0)) * (np.tanh((((data[:,2]) / 2.0)))))) / 2.0)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) * (((((data[:,11]) / 2.0)) / 2.0)))))
    
    def GP_class_9(self,data):
        return(0.250000*np.tanh(((((((data[:,14]) * 2.0)) / 2.0)) + (((((data[:,14]) + (np.tanh((((data[:,14]) * 2.0)))))) + (data[:,14]))))) +
                0.250000*np.tanh(((data[:,9]) - (((((-1.0*((data[:,11])))) + (data[:,9]))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) * 2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,14]))/2.0)) + ((((data[:,13]) + ((((np.tanh((((data[:,9]) + ((6.0)))))) + (data[:,9]))/2.0)))/2.0)))) +
                0.250000*np.tanh(np.tanh(((1.0)))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((np.tanh((((data[:,20]) * (((((((np.tanh((((data[:,20]) / 2.0)))) + (data[:,12]))/2.0)) + ((((data[:,15]) + ((((data[:,17]) + (data[:,9]))/2.0)))/2.0)))/2.0)))))) + (data[:,9]))/2.0)))))) / 2.0)) +
                0.250000*np.tanh(((((data[:,2]) - (data[:,11]))) / 2.0)) +
                0.250000*np.tanh((((((((data[:,13]) / 2.0)) + (((data[:,7]) + (data[:,7]))))/2.0)) - (((((data[:,22]) + (((((data[:,14]) * (((((data[:,7]) * (((data[:,13]) - ((((data[:,7]) + (((((data[:,13]) / 2.0)) + (data[:,7]))))/2.0)))))) / 2.0)))) * 2.0)))) / 2.0)))) +
                0.250000*np.tanh(((((((((((((((data[:,1]) / 2.0)) + ((1.0)))/2.0)) + ((1.0)))/2.0)) + ((((((((((1.0)) / 2.0)) + (np.tanh(((1.0)))))/2.0)) + (np.tanh((((((1.0)) + ((1.0)))/2.0)))))/2.0)))/2.0)) + ((((data[:,20]) + ((1.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,1]) * ((((data[:,14]) + (np.tanh(((((0.0)) / 2.0)))))/2.0)))))
    
    def GP_class_10(self,data):
        return(0.250000*np.tanh((((((((((data[:,2]) + (((data[:,9]) * (np.tanh((data[:,14]))))))/2.0)) + ((((((data[:,14]) + (data[:,20]))/2.0)) + (data[:,0]))))/2.0)) + (data[:,14]))/2.0)) +
                0.250000*np.tanh((((((data[:,9]) + ((-1.0*((((((((2.04309988021850586)) + (((data[:,11]) + (((np.tanh((data[:,0]))) / 2.0)))))/2.0)) * 2.0))))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh(data[:,21]) +
                0.250000*np.tanh((((((((data[:,14]) + (np.tanh((((data[:,6]) / 2.0)))))/2.0)) + (data[:,14]))) + (((((data[:,14]) * 2.0)) + ((((data[:,13]) + (data[:,14]))/2.0)))))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((((((data[:,15]) * 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((2.72836518287658691)) * ((((1.0)) / 2.0)))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((data[:,7]) + (np.tanh(((((((((((0.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))))))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(np.tanh((((np.tanh(((11.43236827850341797)))) / 2.0)))) +
                0.250000*np.tanh(np.tanh((((data[:,7]) / 2.0)))))
    
    def GP_class_11(self,data):
        return(0.250000*np.tanh((-1.0*((((((((((((((data[:,5]) - (((data[:,9]) - ((((-1.0*(((11.40053558349609375))))) / 2.0)))))) + ((((11.40053558349609375)) + ((((11.40053558349609375)) * 2.0)))))) / 2.0)) - (data[:,7]))) - ((-1.0*(((11.40053939819335938))))))) * 2.0))))) +
                0.250000*np.tanh(((((data[:,14]) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((data[:,2]) + (((((data[:,5]) + (data[:,3]))) + ((((data[:,2]) + (((data[:,3]) * 2.0)))/2.0)))))) +
                0.250000*np.tanh(((((((((((data[:,21]) + (data[:,21]))) + ((-1.0*(((((np.tanh((np.tanh((((data[:,14]) / 2.0)))))) + (data[:,21]))/2.0))))))/2.0)) * 2.0)) + ((((data[:,14]) + (data[:,3]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((data[:,0]) + ((((data[:,9]) + ((((data[:,9]) + (((((((((((((((data[:,9]) / 2.0)) - (data[:,10]))) + (data[:,5]))/2.0)) - (data[:,9]))) + (data[:,9]))/2.0)) / 2.0)))/2.0)))/2.0)))/2.0)) + (data[:,9]))/2.0)) +
                0.250000*np.tanh((((((((data[:,13]) + ((((data[:,13]) + (data[:,13]))/2.0)))) + (data[:,1]))/2.0)) / 2.0)) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((data[:,2]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((data[:,9]) + (data[:,9]))/2.0)) +
                0.250000*np.tanh(data[:,2]))
    
    def GP_class_12(self,data):
        return(0.250000*np.tanh((((((((-1.0*((data[:,18])))) - (np.tanh((data[:,9]))))) + ((8.0)))) - ((14.74865913391113281)))) +
                0.250000*np.tanh((((((data[:,21]) + (np.tanh((data[:,9]))))) + ((((((((data[:,5]) / 2.0)) * 2.0)) + (data[:,5]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) + (data[:,14]))) + (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh(((data[:,2]) + (((((((data[:,14]) + ((((np.tanh((data[:,10]))) + (data[:,17]))/2.0)))/2.0)) + (data[:,21]))/2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((((((((data[:,9]) + ((((((data[:,9]) / 2.0)) + (((data[:,9]) * 2.0)))/2.0)))/2.0)) + ((-1.0*((data[:,21])))))/2.0)) * (data[:,9]))) + (((((np.tanh((((data[:,21]) / 2.0)))) / 2.0)) / 2.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((((0.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh((((((((data[:,2]) / 2.0)) * 2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) * 2.0)) + (np.tanh((data[:,14]))))/2.0)) +
                0.250000*np.tanh(np.tanh((((((((((-1.0*((((np.tanh((data[:,9]))) / 2.0))))) / 2.0)) / 2.0)) + (((data[:,3]) * (data[:,13]))))/2.0)))) +
                0.250000*np.tanh((-1.0*((((((((-1.0*(((((((-1.0*((data[:,18])))) - ((((0.0)) / 2.0)))) / 2.0))))) + (np.tanh(((((-1.0*(((-1.0*((data[:,22]))))))) / 2.0)))))/2.0)) / 2.0))))))
    
    def GP_class_13(self,data):
        return(0.250000*np.tanh(((((np.tanh((np.tanh((data[:,2]))))) - ((10.0)))) - (np.tanh((((((data[:,7]) - ((((((6.0)) - ((4.87243366241455078)))) - ((9.0)))))) - ((14.80352973937988281)))))))) +
                0.250000*np.tanh(((np.tanh((((((3.0)) + (data[:,1]))/2.0)))) - ((7.0)))) +
                0.250000*np.tanh(((((np.tanh((np.tanh((np.tanh((data[:,6]))))))) * 2.0)) - ((6.10337877273559570)))) +
                0.250000*np.tanh(((((((data[:,9]) + (data[:,14]))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((np.tanh(((((-1.0*(((((((data[:,2]) + ((10.0)))/2.0)) / 2.0))))) - ((13.28130435943603516)))))) + ((10.0)))) - ((13.28130435943603516)))) + ((-1.0*((data[:,5])))))) +
                0.250000*np.tanh(((((((((((data[:,20]) + (((((((((data[:,20]) * 2.0)) * 2.0)) * 2.0)) + (data[:,13]))))/2.0)) + (((data[:,9]) + (((data[:,9]) + (((data[:,6]) + (data[:,9]))))))))/2.0)) + (data[:,20]))) + (data[:,15]))) +
                0.250000*np.tanh(((((((((data[:,14]) / 2.0)) + ((((data[:,9]) + (((data[:,15]) + (((data[:,5]) + ((((((data[:,9]) + (((data[:,5]) + (data[:,14]))))) + (data[:,9]))/2.0)))))))/2.0)))) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh((((data[:,13]) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((data[:,9]) + ((((data[:,3]) + ((((data[:,21]) + (data[:,3]))/2.0)))/2.0)))) +
                0.250000*np.tanh((((data[:,2]) + (((((((((((((((data[:,20]) + (data[:,1]))) + (((data[:,2]) + (data[:,14]))))) + (data[:,2]))) + (np.tanh((data[:,2]))))/2.0)) + (((data[:,14]) + (data[:,2]))))) + (data[:,14]))/2.0)))/2.0)))
    
    def GP_class_14(self,data):
        return(0.250000*np.tanh((((((((((((5.54024744033813477)) - ((((((9.0)) * 2.0)) * 2.0)))) - (data[:,0]))) - ((((7.0)) + ((4.74105215072631836)))))) - (((((7.0)) + (((data[:,5]) * 2.0)))/2.0)))) - ((9.0)))) +
                0.250000*np.tanh(((((((((9.0)) - ((14.27298927307128906)))) - (data[:,0]))) + (np.tanh(((((12.77708148956298828)) - ((14.80560111999511719)))))))/2.0)) +
                0.250000*np.tanh(((((((0.0)) + ((-1.0*((((data[:,22]) + (((((((((13.30077362060546875)) - (np.tanh((data[:,4]))))) + (((data[:,22]) / 2.0)))/2.0)) / 2.0))))))))/2.0)) - (((np.tanh((((data[:,7]) - (data[:,7]))))) / 2.0)))) +
                0.250000*np.tanh(((((((data[:,6]) + (data[:,14]))/2.0)) + (((np.tanh((((((data[:,14]) + (data[:,14]))) + (((data[:,6]) - (((data[:,5]) + (((np.tanh((data[:,21]))) + ((((data[:,14]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)))))))))))) + (data[:,5]))))/2.0)) +
                0.250000*np.tanh((((((((((((((((((data[:,2]) + (data[:,16]))) + ((((((data[:,2]) / 2.0)) + (data[:,9]))/2.0)))/2.0)) + (np.tanh((data[:,10]))))/2.0)) + (data[:,9]))) + (data[:,9]))/2.0)) + (data[:,21]))) + (data[:,3]))) +
                0.250000*np.tanh((((data[:,13]) + (((((((((((data[:,3]) / 2.0)) / 2.0)) / 2.0)) * (data[:,3]))) / 2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (((((((data[:,14]) / 2.0)) + (data[:,14]))) * 2.0)))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((data[:,1]) / 2.0)) +
                0.250000*np.tanh((((((data[:,2]) + ((((((data[:,13]) + (data[:,14]))) + (data[:,13]))/2.0)))) + (np.tanh((data[:,2]))))/2.0)))
    
    def GP_class_15(self,data):
        return(0.250000*np.tanh((((((((4.0)) + ((((8.0)) / 2.0)))) - ((9.56193733215332031)))) - (((((np.tanh(((13.93556308746337891)))) - (data[:,18]))) + ((13.93556308746337891)))))) +
                0.250000*np.tanh(((data[:,4]) - ((((12.76094818115234375)) - (data[:,8]))))) +
                0.250000*np.tanh((((-1.0*(((12.35446166992187500))))) - (((((12.35446166992187500)) + ((((((12.35446166992187500)) - ((-1.0*(((0.0))))))) - (data[:,0]))))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) - ((14.46367263793945312)))) +
                0.250000*np.tanh(((((data[:,2]) + ((((data[:,9]) + ((-1.0*(((-1.0*(((-1.0*((((data[:,7]) * (((data[:,21]) * 2.0))))))))))))))/2.0)))) + (data[:,9]))) +
                0.250000*np.tanh(((((((((((data[:,7]) + (data[:,3]))/2.0)) + (data[:,20]))/2.0)) - ((((2.33354020118713379)) / 2.0)))) - ((((2.0)) + ((8.78930377960205078)))))) +
                0.250000*np.tanh(((((((data[:,14]) + (((((((data[:,14]) + (((data[:,15]) + (data[:,14]))))) + (((data[:,15]) + (data[:,14]))))) + (((data[:,14]) / 2.0)))))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((((data[:,13]) + (data[:,13]))) + (((data[:,13]) + (data[:,12]))))) + (data[:,9]))) + ((((((data[:,9]) + (((data[:,13]) + ((((data[:,9]) + (((((data[:,13]) * 2.0)) / 2.0)))/2.0)))))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((data[:,9]) + (data[:,9]))/2.0)) / 2.0)))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,8]))) + (data[:,14]))/2.0)))
    
    def GP_class_16(self,data):
        return(0.250000*np.tanh(((((((((((((9.37592792510986328)) - ((13.36316204071044922)))) + ((11.89122962951660156)))/2.0)) - ((((9.0)) - (((((((data[:,13]) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,4]) / 2.0))))) * 2.0)))))))) - ((12.24639320373535156)))) - ((11.89122581481933594)))) +
                0.250000*np.tanh(((((np.tanh(((4.51821422576904297)))) - ((13.23712635040283203)))) - ((((((13.23712635040283203)) + (((data[:,22]) - ((((11.40539550781250000)) - (((((9.0)) + ((((((((data[:,7]) + (((data[:,17]) / 2.0)))/2.0)) * 2.0)) / 2.0)))/2.0)))))))) - (data[:,5]))))) +
                0.250000*np.tanh(((data[:,14]) - ((((4.0)) + (((((13.41770648956298828)) + ((12.42538261413574219)))/2.0)))))) +
                0.250000*np.tanh((((5.0)) - ((9.27778816223144531)))) +
                0.250000*np.tanh(((((data[:,16]) - (((((((data[:,1]) - (((np.tanh((((((((9.29507255554199219)) - (((((((14.34461498260498047)) * 2.0)) + (((((7.0)) + ((9.10683441162109375)))/2.0)))/2.0)))) + ((4.21145153045654297)))/2.0)))) * 2.0)))) - ((9.10683441162109375)))) * 2.0)))) - ((((14.34461498260498047)) * 2.0)))) +
                0.250000*np.tanh((((((((((data[:,17]) / 2.0)) + (((data[:,13]) + (data[:,9]))))/2.0)) + (((data[:,17]) + (data[:,9]))))) + (data[:,0]))) +
                0.250000*np.tanh(((data[:,13]) + (((data[:,14]) - (data[:,21]))))) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,5]))/2.0)) + (((((((data[:,9]) / 2.0)) + (data[:,14]))) / 2.0)))) +
                0.250000*np.tanh((((((data[:,17]) + (data[:,9]))) + (((((data[:,17]) / 2.0)) + ((((((((((data[:,9]) + (data[:,9]))) - ((3.0)))) + (data[:,7]))) + (data[:,9]))/2.0)))))/2.0)) +
                0.250000*np.tanh(np.tanh(((((((((((((((data[:,2]) + (((((((((((data[:,22]) + (data[:,21]))/2.0)) + (data[:,2]))/2.0)) * 2.0)) + ((((((data[:,21]) * 2.0)) + (data[:,21]))/2.0)))))) + (data[:,2]))/2.0)) + (data[:,17]))/2.0)) + (np.tanh((data[:,8]))))/2.0)) + (data[:,2]))))))
    
    def GP_class_17(self,data):
        return(0.250000*np.tanh(((((data[:,14]) - ((((data[:,5]) + (((((14.43326377868652344)) + ((6.0)))/2.0)))/2.0)))) - (((((((10.0)) + ((14.84462165832519531)))/2.0)) - ((((7.0)) + (data[:,13]))))))) +
                0.250000*np.tanh((((((((((data[:,14]) + (((((((((data[:,11]) - ((9.71331787109375000)))) / 2.0)) + ((9.0)))) - ((3.0)))))/2.0)) - ((10.0)))) - ((((10.0)) + ((((9.0)) - ((((((10.0)) - ((9.0)))) / 2.0)))))))) - ((10.0)))) +
                0.250000*np.tanh((((((((((8.0)) - ((12.78277492523193359)))) - (data[:,10]))) - (np.tanh(((((((7.0)) - ((14.95321178436279297)))) - ((8.0)))))))) - ((14.95321178436279297)))) +
                0.250000*np.tanh((((7.45682907104492188)) - ((14.98023796081542969)))) +
                0.250000*np.tanh(((((((((((7.0)) + (((((np.tanh((((((((8.0)) * (data[:,8]))) + (data[:,11]))/2.0)))) / 2.0)) - ((8.0)))))/2.0)) - (data[:,20]))) - ((((8.0)) * 2.0)))) + ((8.0)))) +
                0.250000*np.tanh(((data[:,8]) + (((data[:,4]) + (((np.tanh((data[:,4]))) + (((data[:,13]) + (data[:,13]))))))))) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((((((((data[:,14]) + (np.tanh((((data[:,3]) * (data[:,13]))))))) + (data[:,14]))) + (data[:,0]))) + (data[:,8]))) / 2.0)) +
                0.250000*np.tanh((((((data[:,9]) + (np.tanh(((((((data[:,19]) * (((((((data[:,1]) + (((((((data[:,21]) + (data[:,9]))/2.0)) + (data[:,6]))/2.0)))/2.0)) + (data[:,17]))/2.0)))) + (((data[:,15]) / 2.0)))/2.0)))))/2.0)) * 2.0)) +
                0.250000*np.tanh(((data[:,1]) + (((((data[:,3]) + (data[:,15]))) + (np.tanh(((((data[:,20]) + (((data[:,1]) * 2.0)))/2.0)))))))))
    
    def GP_class_18(self,data):
        return(0.250000*np.tanh((((((((((12.59485149383544922)) - ((12.59485149383544922)))) - ((12.59485149383544922)))) - ((((11.47756862640380859)) / 2.0)))) - ((12.59485149383544922)))) +
                0.250000*np.tanh((((((-1.0*(((11.26121807098388672))))) - (((((((((11.33140659332275391)) - ((-1.0*(((9.76293182373046875))))))) - (((((11.33140659332275391)) + ((8.0)))/2.0)))) + ((9.76293182373046875)))/2.0)))) - (data[:,11]))) +
                0.250000*np.tanh(((((((((((3.40250444412231445)) - (np.tanh(((14.86789989471435547)))))) - ((14.86789989471435547)))) + ((((((data[:,5]) + ((-1.0*(((((14.86789989471435547)) * 2.0))))))/2.0)) - ((14.86789989471435547)))))/2.0)) - (((((14.86789989471435547)) + (((((7.0)) + ((-1.0*((data[:,16])))))/2.0)))/2.0)))) +
                0.250000*np.tanh(((data[:,18]) - ((11.84332561492919922)))) +
                0.250000*np.tanh(((((np.tanh(((((8.96548557281494141)) * (data[:,12]))))) - ((-1.0*((((data[:,2]) * (((((data[:,4]) - ((-1.0*((data[:,9])))))) - ((9.0))))))))))) - ((8.96548557281494141)))) +
                0.250000*np.tanh(((((((10.0)) - (data[:,12]))) + (((((((((data[:,13]) - ((12.43707370758056641)))) - ((((12.43707370758056641)) - (((data[:,12]) / 2.0)))))) - ((((12.43707370758056641)) - ((((data[:,13]) + ((12.43707370758056641)))/2.0)))))) - ((12.43707370758056641)))))/2.0)) +
                0.250000*np.tanh(((((((np.tanh((((np.tanh((data[:,14]))) + (data[:,13]))))) * 2.0)) + (data[:,14]))) + (data[:,13]))) +
                0.250000*np.tanh(((np.tanh((data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((data[:,9]) + (((((data[:,13]) + (data[:,4]))) + (data[:,13]))))) / 2.0)) +
                0.250000*np.tanh(((data[:,15]) - ((((data[:,3]) + ((5.0)))/2.0)))))
    
    def GP_class_19(self,data):
        return(0.250000*np.tanh((((((data[:,13]) + (data[:,13]))/2.0)) + ((((((data[:,8]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,3]) + (((data[:,8]) + (np.tanh(((((((-1.0*((((((9.05535793304443359)) + ((((data[:,14]) + (((((((data[:,1]) - (((data[:,12]) * 2.0)))) * ((-1.0*((data[:,17])))))) / 2.0)))/2.0)))/2.0))))) + (data[:,2]))) * 2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,3]) + (((((data[:,14]) + (data[:,14]))) * 2.0)))) + ((((data[:,14]) + (np.tanh((((data[:,14]) + (np.tanh((data[:,14]))))))))/2.0)))) * 2.0)) + ((((data[:,14]) + (data[:,2]))/2.0)))) +
                0.250000*np.tanh(((((data[:,8]) + (((data[:,4]) + (((data[:,13]) + (data[:,10]))))))) + (((((data[:,13]) + (((((data[:,13]) / 2.0)) + (data[:,9]))))) + (np.tanh(((((data[:,8]) + (data[:,8]))/2.0)))))))) +
                0.250000*np.tanh((((((data[:,19]) + (((((data[:,10]) + ((((data[:,11]) + (data[:,19]))/2.0)))) + (data[:,10]))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh((((((data[:,0]) + (((((((((data[:,0]) + (((((data[:,0]) - (data[:,11]))) / 2.0)))/2.0)) * 2.0)) + ((((((np.tanh((data[:,13]))) + (((data[:,0]) / 2.0)))/2.0)) / 2.0)))/2.0)))/2.0)) * 2.0)) +
                0.250000*np.tanh(((((-1.0*((data[:,15])))) + (((data[:,14]) + (data[:,14]))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,0]) + (data[:,15]))/2.0)) + (((data[:,15]) * (data[:,15]))))/2.0)) - (data[:,11]))) +
                0.250000*np.tanh(data[:,13]) +
                0.250000*np.tanh(((((((((data[:,18]) + ((((data[:,4]) + (data[:,18]))/2.0)))) / 2.0)) + ((((data[:,18]) + (data[:,4]))/2.0)))) * (data[:,4])))    )

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 632)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from kaggle.competitions import nflrush
env = nflrush.make_env()
iter_test = env.iter_test()
gp = GP()
for (test_df, sample_prediction_df) in iter_test:
    basetable = create_features(test_df, deploy=True)
    basetable.drop(['GameId','PlayId'], axis=1, inplace=True)
    scaled_basetable = scaler.transform(basetable)
    
    p = [m.predict(scaled_basetable) for m in models]
    y_pred_nn = (np.array(p)).mean(0)
    #np.expm1(np.log1p(np.array(p)).mean(0))

    y_pred_gp = np.zeros((test_df.shape[0],199))
    ans = gp.GrabPredictions(scaled_basetable)
    y_pred_gp[:,96:96+20] = ans
    
    y_pred = (.6*y_pred_nn+.4*y_pred_gp)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]
    
    preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
    env.predict(preds_df)
    
env.write_submission_file()